# Darknet ONNX to tflite conversion

## Install dependencies

In [ ]:
%pip install onnx2tf tensorflow tf-keras onnx onnx-graphsurgeon ai-edge-litert sng4onnx onnxsim

## Generate int8 quantization calibration data

In [ ]:
%cd /content

In [ ]:
import cv2
import glob
import numpy as np

from importlib import reload
import functions.parse_darknet_cfg as parse_darknet_cfg_mod

DARKNET_CONFIG_PATH = "people-r-people_2025-05-05/people-r-people.cfg"
IMAGES_PATH = "people-r-people_2025-05-05/sample_images"

net, layers = parse_darknet_cfg_mod.parse_darknet_cfg(DARKNET_CONFIG_PATH)

files = glob.glob(f"{IMAGES_PATH}/*.jpg")
images = []
for idx, file in enumerate(files):
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(net["height"], net["width"]))
    img = img.astype(np.float32) / 255.0  # convert to 0.0 - 1.0 scale
    images.append(img)

np.save(file=f"{IMAGES_PATH}/calibdata.npy", arr=np.stack(images, axis=0))

## Convert

In [ ]:
%cd /content/people-r-people_2025-05-05
!onnx2tf -i people-r-people.onnx \
    -oiqt \
    -o people-r-people-tf  \
    -cind "frame" "sample_images/calibdata.npy" "[[[[0,0,0]]]]" "[[[[1,1,1]]]]"
 

## Inference

In [ ]:
%cd /content

In [ ]:
import time
import cv2
import os
import numpy as np
from ai_edge_litert.interpreter import Interpreter, load_delegate  # AI Edge Lite / TFLite Runtime
# from tensorflow.lite import Interpreter  # Uncomment if using full TensorFlow instead

import matplotlib.pyplot as plt
%matplotlib inline

from importlib import reload
import functions.parse_darknet_cfg as parse_darknet_cfg_mod
reload(parse_darknet_cfg_mod)
import functions.preprocess_image as preprocess_image_mod
reload(preprocess_image_mod)
import functions.postprocess_output as postprocess_output_mod
reload(postprocess_output_mod)
import functions.visualize_detections as visualize_detections_mod
reload(visualize_detections_mod)


def load_interpreter(model_path, use_edgetpu=False):
    """
    Load a TensorFlow Lite or EdgeTPU interpreter.
    """
    if use_edgetpu:
        delegates = [load_delegate('libedgetpu.so.1')]
        interpreter = Interpreter(model_path=model_path, experimental_delegates=delegates)
    else:
        interpreter = Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter


def run_inference_tflite(model_path, image, yolo_layers, input_width, input_height, class_names, use_edgetpu=False):
    """
    Run inference on a single image using a TFLite/AI Edge Lite model.
    """
    # Load the model
    interpreter = load_interpreter(model_path, use_edgetpu)

    # Get input & output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape']

    # Preprocess image
    input_data, scale, padding = preprocess_image_mod.preprocess_image(input_width, input_height, image)


    # NCHW -> NCHW
    input_data = np.transpose(input_data, (0, 2, 3, 1))

    if input_details[0]["dtype"] != np.float32:
        quant_scale, quant_zero = input_details[0]['quantization']

        input_data = (input_data.astype(np.float32) / quant_scale + quant_zero).round().astype(input_details[0]["dtype"])

        # TODO: Detect output reversion instead of just assuming it for int8
        yolo_layers = list(reversed(yolo_layers))

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    start_time = time.time()
    interpreter.invoke()
    inference_time = (time.time() - start_time) * 1000  # ms

    def dequant(tensor, details):
        if details["dtype"] != np.float32:
            scale_out, zero_point_out = details['quantization']

            return (tensor.astype(np.float32) - zero_point_out) * scale_out
        
        return tensor

    # Gather outputs (NHWC -> NCHW)
    outputs = [np.transpose(dequant(interpreter.get_tensor(o['index']), o), (0, 3, 1, 2)) for o in output_details]


    # Postprocess    
    print("Postprocessing...")
    detections = postprocess_output_mod.postprocess_output(
        outputs,
        yolo_layers_cfg=yolo_layers,
        input_size=(input_width, input_height),
        conf_threshold=CONF_THRESHOLD,
        iou_threshold=IOU_THRESHOLD,
        scale=scale,
        padding=padding,
        class_names=class_names,
    )

    print(f"Inference completed in {inference_time:.2f} ms")
    return detections


DARKNET_MODEL_BASE_NAME="people-r-people_2025-05-05/people-r-people"
NAMES_FILE=f"{DARKNET_MODEL_BASE_NAME}.names"
CFG_FILE=f"{DARKNET_MODEL_BASE_NAME}.cfg"
# TF_MODEL="people-r-people_2025-05-05/people-r-people-tf/people-r-people_float32.tflite"
TF_MODEL="people-r-people_2025-05-05/people-r-people-tf/people-r-people_full_integer_quant.tflite"
CONF_THRESHOLD=0.25
IOU_THRESHOLD=0.45
IMAGES_PATH="people-r-people_2025-05-05/sample_images"

# Load class names
with open(NAMES_FILE, "r", encoding="utf-8") as f:
    class_names = [line.strip() for line in f if line.strip()]

num_classes = len(class_names)

print(f"✅ Loaded {num_classes} classes")

# Load darknet cfg
net, layers = parse_darknet_cfg_mod.parse_darknet_cfg(CFG_FILE)
input_width=net["width"]
input_height=net["height"]
yolo_layers = [layer for layer in layers if layer['type'] == 'yolo']

for image_name in [img for img in os.listdir(IMAGES_PATH) if img.endswith('.jpg')]:
    image_path = f"{IMAGES_PATH}/{image_name}"

    print(f"Processing image {image_path}")

    image = cv2.imread(image_path)

    print("Running inference...")
    detections = run_inference_tflite(TF_MODEL, image=image, 
                                      yolo_layers=yolo_layers,
                                        input_width=input_width,
                                        input_height=input_height,
        class_names=class_names,
                                      
                                      use_edgetpu=False)


    print("Visualize...")
    visualized = visualize_detections_mod.visualize_detections(
        image=image,
        detections=detections[0],
        class_names=class_names
    )

    plt.imshow(cv2.cvtColor(visualized, cv2.COLOR_BGR2RGB),cmap='gray')
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show()


